In [ ]:
 def explanation_from_feature_importance(self, feature_importance_activity, feature_importance_usage_device, scaler_activity, scaler_usage, device='Tumble Dryer', diagnostics=False):
        self.feature_importance_activity = feature_importance_activity
        self.feature_importance_usage_device= feature_importance_usage_device
        self.scaler_activity = scaler_activity
        self.scaler_usage= scaler_usage
        self.device = device
        self.diagnostics = diagnostics

        # Sentence structure:
        # You have one recommendation for the following device: Dishwasher. Please use it on 21.08.2014 at 00:00.
        sentence = 'We based the recommendation on your past activity and usage of the device. '

        #activity_lags:

        #if feature_importance_activity['activity_lag_24'] or feature_importance_activity['activity_lag_48'] or feature_importance_activity['activity_lag_72'] !=0:
            # check if really active in X_test otherwise put not active
        if self.X_test_activity['activity_lag_24'].iloc[self.best_hour] and self.X_test_activity['activity_lag_48'].iloc[self.best_hour] and self.X_test_activity['activity_lag_72'].iloc[self.best_hour] ==0:
            active_past = 'not'
        else:
            active_past = ''
            # input the activity lag with the strongest feature importance
        FI_lag = np.argmax([feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_24','feature_importance_vals'],
                           feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_48','feature_importance_vals'],
                           feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_72','feature_importance_vals']])
        if FI_lag == 0:
            activity_lag = 'day'
        elif FI_lag == 1:
            activity_lag = 'two days'
        elif FI_lag == 2:
            activity_lag = 'three days'
        else:
            activity_lag = 'three days'

        # weather:
        # to do: instead of feature values include transformed values

        weather_array = np.array([feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'dwpt', 'feature_importance_vals'],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'rhum', 'feature_importance_vals'],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'temp', 'feature_importance_vals'],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wdir', 'feature_importance_vals'],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wspd', 'feature_importance_vals']
                                  ])

        d = {'features': ['dwpt', 'rhum', 'temp', 'wdir', 'wspd'],
             'labels': ['dewing point', 'relative humidity','temperature', 'wind direction', 'windspeed'],
             'feature_importances' : [feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'dwpt', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'rhum', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'temp', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wdir', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wspd', 'feature_importance_vals'].to_numpy()[0]],
             'feature_values' : [self.X_test_activity['dwpt'].iloc[self.best_hour],
                                 self.X_test_activity['rhum'].iloc[self.best_hour],
                                 self.X_test_activity['temp'].iloc[self.best_hour],
                                 self.X_test_activity['wdir'].iloc[self.best_hour],
                                 self.X_test_activity['wspd'].iloc[self.best_hour],
                                 ]

             }
        df = pd.DataFrame(data=d)

        weather1_ind = np.argmax(df['feature_importances'])
        weather1 = df['labels'][weather1_ind]

        value1 = df['feature_values'][weather1_ind]
        #value1 = self.scaler_activity.inverse_transform(df['feature_values'][weather1_ind])
        weather2 = None
        value2= None

        # final activity sentence
        sentence_activity = (f"We believe you are active today since you were {active_past} active during the last {activity_lag}."
                             f"The weather conditions ({weather1}:{value1}, {weather2}:{value2}) support that recommendation.")

        explanation_sentence = sentence + sentence_activity

        print(self.X_test_usage.iloc[self.best_hour])
        #
        # Usage Sentence:
        # %%
        sentence_usage = 'We believe you are likely to use the device in the near future since'
        #  active_last_2_days: wenn da FI wichtig genug ist: if in X_test 0 --> 'you were NOT active the last two days', X_test 1 --> 'you were active the last two days'
        #  'you used the [device] the last [].
        #     Dishwasher_usage_lag_2:
        #     Dishwasher_usage_lag_1

        # 'Additionally, the weather conditions support that recommendation'   #(add strongest weather)
        #                      wdir
        #                       wspd
        #

        explanation_sentence = sentence + sentence_activity + sentence_usage
        # optional vizualizations
        if self.diagnostics == True:
            print('Vizualizations for further insights into our predictions: ')
            print('add plots here')

        return explanation_sentence





